# 📓 PyQGIS Code Snippets

## 0. Environment Check

In [ ]:
from qgis.core import *
from qgis.utils import iface

print(Qgis.QGIS_VERSION)
print(QgsApplication.qgisSettingsDirPath())

## 1. Access QGIS Interface Objects

In [ ]:
canvas = iface.mapCanvas()
project = QgsProject.instance()

## 2. Add Layers

### 2.1 Add Local Vector Layer

In [ ]:
vector_path = "/path/to/data.shp"

vlayer = QgsVectorLayer(vector_path, "My Vector Layer", "ogr")
QgsProject.instance().addMapLayer(vlayer)

print(vlayer.isValid())

### 2.2 Add Raster Layer

In [ ]:
raster_path = "/path/to/image.tif"

rlayer = QgsRasterLayer(raster_path, "My Raster")
QgsProject.instance().addMapLayer(rlayer)

print(rlayer.isValid())

### 2.3 Add Online GeoJSON

In [ ]:
url = (
    "https://github.com/opengeos/datasets/releases/download/world/world_cities.geojson"
)

layer = QgsVectorLayer(url, "World Cities", "ogr")
iface.addVectorLayer(url, "World Cities", "ogr")

### 2.4 Add XYZ Tiles (OpenStreetMap)

In [ ]:
url = "type=xyz&url=https://tile.openstreetmap.org/{z}/{x}/{y}.png"
osm = QgsRasterLayer(url, "OpenStreetMap", "wms")

QgsProject.instance().addMapLayer(osm)

## 3. Zoom & Navigation

### Zoom to Active Layer

In [ ]:
iface.zoomToActiveLayer()

### Zoom to Full Extent

In [ ]:
iface.mapCanvas().zoomToFullExtent()

### Zoom to Selected Features

In [ ]:
iface.mapCanvas().zoomToSelected()

## 4. Layer Inspection

### List All Layers

In [ ]:
for layer in project.mapLayers().values():
    print(layer.name(), layer.type())

### Get Active Layer

In [ ]:
layer = iface.activeLayer()
print(layer.name())

## 5. Feature & Geometry Operations

### Iterate Over Features

In [ ]:
for f in layer.getFeatures():
    print(f.id(), f.attributes())

### Access Geometry

In [ ]:
for f in layer.getFeatures():
    print(f.geometry().asWkt())

### Select by Expression

In [ ]:
layer.selectByExpression("population > 1000000")
layer.selectedFeatureCount()

### Clear Selection

In [ ]:
layer.removeSelection()

## 6. Attribute Table Operations

### Add a Field

In [ ]:
from PyQt5.QtCore import QVariant

layer.startEditing()

layer.addAttribute(QgsField("area_km2", QVariant.Double))
layer.updateFields()

layer.commitChanges()

### Update Attribute Values

In [ ]:
layer.startEditing()

idx = layer.fields().indexOf("area_km2")

for f in layer.getFeatures():
    area = f.geometry().area() / 1e6
    layer.changeAttributeValue(f.id(), idx, area)

layer.commitChanges()

## 7. Styling & Symbology

### Change Single Symbol Color

In [ ]:
from PyQt5.QtGui import QColor

symbol = layer.renderer().symbol()
symbol.setColor(QColor("red"))

layer.triggerRepaint()

### Graduated Renderer

In [ ]:
from qgis.core import QgsGraduatedSymbolRenderer

renderer = QgsGraduatedSymbolRenderer.createRenderer(
    layer,
    "population",
    5,
    QgsGraduatedSymbolRenderer.EqualInterval,
    QgsSymbol.defaultSymbol(layer.geometryType()),
    QgsColorRamp("Blues"),
)

layer.setRenderer(renderer)
layer.triggerRepaint()

## 8. Labeling

In [ ]:
settings = QgsPalLayerSettings()
settings.fieldName = "name"
settings.enabled = True

layer.setLabeling(QgsVectorLayerSimpleLabeling(settings))
layer.setLabelsEnabled(True)

layer.triggerRepaint()

## 9. CRS & Reprojection

### Check CRS

In [ ]:
layer.crs().authid()

### Reproject Layer (In Memory)

In [ ]:
import processing

params = {
    "INPUT": layer,
    "TARGET_CRS": QgsCoordinateReferenceSystem("EPSG:4326"),
    "OUTPUT": "memory:",
}

result = processing.run("native:reprojectlayer", params)
QgsProject.instance().addMapLayer(result["OUTPUT"])

## 10. Processing Algorithms

### Buffer

In [ ]:
params = {"INPUT": layer, "DISTANCE": 1000, "SEGMENTS": 8, "OUTPUT": "memory:"}

buffered = processing.run("native:buffer", params)
QgsProject.instance().addMapLayer(buffered["OUTPUT"])

### Clip

In [ ]:
processing.run(
    "native:clip", {"INPUT": input_layer, "OVERLAY": clip_layer, "OUTPUT": "memory:"}
)

## 11. Raster Operations

### RGB Band Combination

In [ ]:
renderer = QgsMultiBandColorRenderer(rlayer.dataProvider(), 4, 3, 2)

rlayer.setRenderer(renderer)
rlayer.triggerRepaint()

### Contrast Stretch

In [ ]:
renderer.setContrastEnhancement(QgsContrastEnhancement.StretchToMinimumMaximum)
rlayer.triggerRepaint()

## 12. Export Data

### Export Selected Features

In [ ]:
QgsVectorFileWriter.writeAsVectorFormat(
    layer, "/path/output.geojson", "UTF-8", layer.crs(), "GeoJSON", onlySelected=True
)

### Export Map as Image

In [ ]:
from PyQt5.QtCore import QSize

settings = QgsMapSettings()
settings.setLayers([layer])
settings.setExtent(layer.extent())
settings.setOutputSize(QSize(1920, 1080))

job = QgsMapRendererParallelJob(settings)
job.start()
job.waitForFinished()

img = job.renderedImage()
img.save("/path/map.png")

## 13. Messages & Logging

In [ ]:
iface.messageBar().pushInfo("PyQGIS", "Task completed successfully")

In [ ]:
QgsMessageLog.logMessage("Debug message", "Notebook", Qgis.Info)

## 14. Plugin-Friendly Utilities

In [ ]:
from qgis.utils import iface

In [ ]:
QgsApplication.instance().messageLog().messageReceived.connect(
    lambda msg, tag, level: print(msg)
)